In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy import fftpack

%matplotlib inline

In [ ]:
img=mpimg.imread('bossier.jpg')
print img.shape, type(img)

In [ ]:
print img.shape[0]/8.0, img.shape[1]/8.0
print (img.shape[0]+1)/8.0, (img.shape[1]-2)/8.0

In [ ]:
r = img.shape[0]+1
c = img.shape[1]-2
img2 = np.zeros(r*c*3, dtype=np.dtype(np.uint8)).reshape(r,c,3)
img2[:975,:728,:] = img[:975,:728,:]
img = img2

In [ ]:
imgplot = plt.imshow(img)

In [ ]:
def get_2D_dct(img):
    """ Get 2D Cosine Transform of Image
    """
    d = fftpack.dct(fftpack.dct(img.T, norm='ortho').T, norm='ortho')
    return d.astype(int)

def get_2D_idct(coefficients):
    """ Get 2D Inverse Cosine Transform of Image
    """
    id = fftpack.idct(fftpack.idct(coefficients.T, norm='ortho').T, norm='ortho')
    return id.astype(int)

def get_8x8_block(img, r_start, c_start, channel):
    """ Get an 8x8 block from the image
    """
    return img[r_start:r_start+8, c_start:c_start+8, channel]

def put_8x8_block(img, block, r_start, c_start, channel):
    img_new = img
    img_new[r_start:r_start+8, c_start:c_start+8, channel] = block
    return img_new

def quantize(dct_block, factor):
    """ Get a quantized version of a matrix
    """
    f = float(factor)
    q = np.floor(dct_block/f)*f
    return q.astype(int)

def compress(img, factor):
    img_reconstructed = np.zeros_like(img)

    for start_row in np.arange(0,img.shape[0],8):
        for start_col in np.arange(0,img.shape[1],8):
            for channel in range(0,3):
                block = get_8x8_block(img, start_row, start_col, channel)
                block_dct = get_2D_dct(block)
                block_dct_quantized = quantize(block_dct, factor)
                block_reconstructed = get_2D_idct(block_dct_quantized)
                img_reconstructed = put_8x8_block(img_reconstructed, block_reconstructed, start_row, start_col, channel)
    return img_reconstructed

In [ ]:
factor = 16


## Converting to YCbCr

https://en.wikipedia.org/wiki/YCbCr

JFIF usage of JPEG allows Y′CbCr where Y′, CB and CR have the full 8-bit range of 0-255:[4]

$$
    \begin{align} Y' &=& 0 &+ (0.299 & \cdot R'_D) &+ (0.587 & \cdot G'_D) &+ (0.114 & \cdot B'_D)\\ C_B &=& 128 & - (0.168736 & \cdot R'_D) &- (0.331264 & \cdot G'_D) &+ (0.5 & \cdot B'_D)\\ C_R &=& 128 &+ (0.5 & \cdot R'_D) &- (0.418688 & \cdot G'_D) &- (0.081312 & \cdot B'_D) \end{align}
$$

And back:

$$
    \begin{align} R &=& Y &&& + 1.402 & \cdot (C_R-128) \\ G &=& Y & - 0.34414 & \cdot (C_B-128)& - 0.71414 & \cdot (C_R-128) \\ B &=& Y & + 1.772 & \cdot (C_B-128)& \end{align} 
$$

Factor out bias terms:

$$
\begin{align}
R &= 1 \times Y + 0 \times (C_B - 128) + 1.402 \times (C_R-128) \\
R &= 1 \times Y + 0 C_B - 0 \times 128 + 1.402 C_R - 1.402 \times 128 \\
R &= - 1.402 \times 128 + 1 \times Y + 0 \times C_B + 1.402 \times C_R \\
R &= - 179.456 + 1 \times Y + 0 \times C_B + 1.402 \times C_R \\
\\
G &= 1 \times Y - 0.34414 \times (C_B - 128) - 0.71414 \times (C_R-128) \\
G &= 1 \times Y - 0.34414 \times C_B + 0.34414 \times 128 - 0.71414 \times C_R + 0.71414 \times 128 \\
G &= (0.34414 + 0.71414) \times 128 + 1 \times Y - 0.34414 \times C_B  - 0.71414 \times C_R \\
G &= 135.45984 + 1 \times Y - 0.34414 \times C_B  - 0.71414 \times C_R \\
\\
B &= 1 \times Y + 1.772 \times (C_B - 128) + 0 \times (C_R-128) \\
B &= 1 \times Y + 1.772 \times C_B - 1.772 \times 128 + 0 \times C_R \\
B &= - 1.772 \times 128 + 1 \times Y + 1.772 \times C_B + 0 \times C_R \\
B &= - 226.816 + 1 \times Y + 1.772 \times C_B + 0 \times C_R \\
\end{align}
$$



In [ ]:
def rgb2ycbcr(img):
    img_new = np.zeros_like(img)
    CM_to_ycbcr = np.array([[0, .299, .587, .114],[128, -.168736, -.331264, .5],[128, .5, -.418688, -.081312]])
    
    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            pix_with_bias = np.ones(4)
            pix_with_bias[1:4] = img[row, col, :]
            img_new[row, col, :] = np.dot(CM_to_ycbcr, pix_with_bias)
    return img_new

def ycbcr2rgb(img):
    img_new = np.zeros_like(img)
    CM_to_rgb = np.array([[-179.456, 1, 0, 1.402],[135.45984, 1, -.34414, -.71414],[-226.816, 1, 1.772, 0]])

    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            pix_with_bias = np.ones(4)
            pix_with_bias[1:4] = img[row, col, :]
            img_new[row, col, :] = np.dot(CM_to_rgb, pix_with_bias)
    return img_new

In [ ]:
factor = 64

In [ ]:
img_compressed = compress(img, factor)
imgplot = plt.imshow(img_compressed)

In [ ]:
img_ycbcr = rgb2ycbcr(img)
img_ycbcr_compressed = compress(img_ycbcr, factor)
img_rgb_compressed = ycbcr2rgb(img_ycbcr_compressed)

imgplot = plt.imshow(img_rgb_compressed)

In [ ]:
fname = "b_rgb_" + str(factor) + ".jpg"
mpimg.imsave(fname, img_compressed)
fname = "b_ycbcr_" + str(factor) + ".jpg"
mpimg.imsave(fname, img_rgb_compressed)